## 示例选择器
- 根据长度要求智能选择  字符长度
- 根据输入相似度选择示例 最大边际相似度
- 根据输入相似度选择    最大余弦相似度

In [19]:
# 根据输入的提示词长度智能选择合适的示例数据
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

# 定义示例数据
examples = [
  {"input":"happy","output":"sad"},
  {"input":"sad","output":"happy"},
  {"input":"tall","output":"short"},
  {"input":"short","output":"tall"},
  {"input":"angry","output":"excited"},
  {"input":"excited","output":"angry"},
  {"input":"开心","output":"悲伤"},
  {"input":"悲伤","output":"开心"},
]

# 创建提示词模板
example_prompt = PromptTemplate(
  input_variables=["input","output"],
  template="原词: {input} \n反义: {output}",
)

# 调用长度示例选择器
example_selector = LengthBasedExampleSelector(
  # 传入示例
  examples = examples,
  # 传入提示词模板
  example_prompt = example_prompt,
  # 设置最大示例长度
  max_length=20,
)

# 使用小样本提示词模板来实现动态选择示例
few_shot_example = FewShotPromptTemplate(
  # 示例选择器
  example_selector=example_selector,
  # 示例模板
  example_prompt=example_prompt,
  # priex
  prefix="给出每一个输入词的反义词: ",
  # suffix
  suffix="原词: {adjective} \n反义: ",
  input_variables = ["adjective"]
)

# 生成提示词 格式化输入词
prompt = few_shot_example.format(adjective="happy")
print(prompt)




给出每一个输入词的反义词: 

原词: happy 
反义: sad

原词: sad 
反义: happy

原词: tall 
反义: short

原词: happy 
反义: 


示例选择器中的输入相似度选择和最大边际相关性（MMR）是两种用于优化示例选择的方法，其核心目标是在保持相关性的同时提升多样性。以下是具体解释：

### 1. **输入相似度选择**
输入相似度选择通过计算用户输入与示例之间的语义相似度（如余弦相似度）来筛选最相关的示例。具体步骤包括：
- **嵌入生成**：使用预训练模型（如OpenAIEmbeddings）将输入和示例转换为向量表示。
- **相似度计算**：通过向量间的余弦相似度衡量语义关联度，选择相似度最高的示例。
- **局限性**：仅依赖相似度可能导致选择的示例过于集中，缺乏多样性。

### 2. **最大边际相关性（MMR）**
MMR在相似度选择的基础上引入多样性控制，其核心思想是**平衡相关性与多样性**。具体机制如下：
- **迭代选择**：每次选择一个新示例时，最大化该示例与已选示例集的边际相关性（即新示例对整体相关性的贡献）。
- **惩罚相似性**：对已选示例的相似度进行惩罚（如减去相似度分数），避免重复选择相似内容。
- **公式化表达**：  
  \其中，\( s(x_i, x) \)为输入与示例的相似度，\( \lambda \)为多样性权重参数（控制相似度与多样性的平衡）。

### 3. **应用场景与优势**
- **场景**：适用于需要高精度且需覆盖多角度的生成任务（如对话、摘要）。
- **优势**：  
  - 相比纯相似度选择，MMR能生成更自然的提示，减少重复性输出。  
  - 通过动态调整多样性权重，可灵活适应不同任务需求。

### 4. **代码实现示例**
在LangChain中，可通过`MaxMarginalRelevanceExampleSelector`实现MMR：
```python
from langchain.prompts import MaxMarginalRelevanceExampleSelector

example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    examples,  # 示例数据
    OpenAIEmbeddings(),  # 嵌入模型
    FAISS,  # 向量数据库
    k=3,  # 选择示例数
    diversity_weight=0.5  # 多样性权重
)
```
此代码会从示例中选择3个既相关又多样的样本。

### 总结
输入相似度选择侧重单一相关性，而MMR通过惩罚机制和迭代优化，在相关性基础上增强多样性，是更全面的示例选择策略。实际应用中需根据任务需求调整参数（如`diversity_weight`）以平衡效果。

In [ ]:
# ! pip3 install tiktoken
# ! pip3 install faiss-cpu
# ! pip3 install langchain_community -U
# ! pip3 install pinecone

  Using cached faiss_cpu-1.10.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (4.4 kB)
Using cached faiss_cpu-1.10.0-cp312-cp312-macosx_11_0_arm64.whl (3.2 MB)


In [2]:
from langchain.prompts.example_selector import MaxMarginalRelevanceExampleSelector
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
# from langchain.vectorstores import Pinecone
# from langchain.embeddings import OllamaEmbeddings
from langchain_ollama import OllamaEmbeddings
from langchain.prompts import FewShotPromptTemplate

# faiss——CPU 和 OpenMP 冲突问题
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

OLLAMA_URL = "http://localhost:11434"  # Ollama 默认地址
API_KEYS = {"OLLAMA_DEEPSEEK_API_KEY"}     # 自定义合法 API Key


# 定义示例数据
examples = [
  {"input":"happy","output":"sad"},
  {"input":"sad","output":"happy"},
  {"input":"tall","output":"short"},
  {"input":"short","output":"tall"},
  {"input":"angry","output":"excited"},
  {"input":"excited","output":"angry"},
  {"input":"开心","output":"悲伤"},
  {"input":"悲伤","output":"开心"},
]

# 创建提示词模板
example_prompt = PromptTemplate(
  input_variables=["input","output"],
  template="原词: {input} \n反义: {output}",
)

# 调用 Ollama API
embed = OllamaEmbeddings(
    model="deepseek-r1:1.5b",
    base_url=OLLAMA_URL,
)
# 创建一个示例选择器
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
  # 传入示例
  examples=examples,
  # 相似性搜索
  embeddings=embed,
  # 使用的向量存储
  vectorstore_cls=FAISS,
  # 结果条数
  k=2,
)

mmr_prompt = FewShotPromptTemplate(
  example_selector=example_selector,
  example_prompt=example_prompt,
  prefix="给出每一个输入词的反义词: ",
  suffix="原词: {adjective} \n反义: ",
  input_variables=["adjective"]
)

# 当我们输入一个词时，我们会得到一个提示，提示我们输入这个词的反义词
prompt = mmr_prompt.format(adjective="担心")
print(prompt)

# 执行链式调用
from langchain_ollama import ChatOllama
# 指定本地模型名称或路径
ollama_llm = ChatOllama(
    model="deepseek-r1:1.5b",  # 例如DeepSeek-R1 1.5B模型
    base_url="http://localhost:11434"  # Ollama默认地址
)
response = ollama_llm.invoke(prompt)
print(response)


给出每一个输入词的反义词: 

原词: 开心 
反义: 悲伤

原词: short 
反义: tall

原词: 担心 
反义: 
content='<think>\n嗯，我现在要找出“担忧”这个词的反义词。首先，“担忧”是什么意思呢？它通常是指认为事情会变糟或者不容易成功，比如在工作上、生活中感到压力，可能会担心 things会不好 happen。\n\n那么，反义词是什么呢？反义词就是指相对应的词语，意思相反的意思。“担忧”和“担忧”本身已经是一个表达方式了，可能需要找到另一个更具体的表达。常见的反义词有“安心”，意思是安全、舒适的感觉；“平和”则指的是平静、不紧张的情绪。\n\n再想想，“担忧”还有其他含义吗？比如在某些情况下，可能指对某种可能性的不确定感到担心，比如关于考试成绩的事情，可能会“担忧”自己不会考好。那么反义词也应该能对应这种状态。“安心”确实适合，因为它表示一种平静和安全的感觉，与担忧相反。\n\n有没有其他的可能性呢？比如“不安”，这通常是指不安分、不确定的情绪，可能有些接近，“担忧”和“不安”都带有不确定的意味，但“不安”更偏向于对不确定性感到不确定，而“担忧”更多的是对事情可能性的担忧。所以，反义词中比较合适的是“安心”。\n\n再查一下是否有其他词可以考虑。比如“忧虑”，它指的是担心事情会变得更糟，听起来有些接近，“担忧”的意思就是一种负面情绪，和忧虑不同。所以，可能需要选择最相近且准确的词汇。\n\n综上所述，“担忧”反义应该是“安心”。这样就能很好地表达出与“担忧”相反的情绪状态了。\n</think>\n\n反义词：安安心心' additional_kwargs={} response_metadata={'model': 'deepseek-r1:1.5b', 'created_at': '2025-03-12T03:36:05.960269Z', 'done': True, 'done_reason': 'stop', 'total_duration': 7778197209, 'load_duration': 27399625, 'prompt_eval_count': 49, 'prompt_eval_duration': 84000000, 'eval_count': 341, 'eval_duration': 7662

## 最大余弦相似度
 - 一种常见的余弦相似度计算方法<br>
余弦相似度是一种常见的衡量两个向量之间相似度的方法，通常用于文本分析和推荐系统中。它通过计算两个向量的夹角余弦值来表示它们的相似度，值域在[-1, 1]之间，其中1表示完全相同，-1表示完全相反，0表示两者正交（无相似性）。
 - 重点关注如何计算两个向量之间的相似度 

In [3]:
import numpy as np

def cosine_similarity(vec1, vec2):
    """
    计算两个向量之间的余弦相似度
    :param vec1: 向量1
    :param vec2: 向量2
    :return: 余弦相似度
    """
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

# 示例向量
vector1 = np.array([1, 2, 3])
vector2 = np.array([4, 5, 6])

# 计算余弦相似度
similarity = cosine_similarity(vector1, vector2)
print(f"两个向量之间的余弦相似度: {similarity}")

两个向量之间的余弦相似度: 0.9746318461970762


In [4]:
! pip3 install chromadb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 1.5 MB/s eta 0:00:00-:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 3.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 5.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.7/33.7 MB 6.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 6.7 MB/s eta 0:00:00
  Created wheel for pypika: filename=pypika-0.48.9-py2.py3-none-any.whl size=53769 sha256=162f3208ed314109f9251e6e666de483df970ac950a659643298223b7ea5592e
  Stored in directory: /Users/baisongtao/Library/Caches/pip/wheels/d5/3d/69/8d68d249cd3de2584f226e27fd431d6344f7d70fd856ebd01b
Successfully built pypika
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib_metadata 8.6.1
    Uninstalling importlib_metada

In [5]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
# from langchain.vectorstores import Pinecone
# from langchain.embeddings import OllamaEmbeddings
from langchain_ollama import OllamaEmbeddings
from langchain.prompts import FewShotPromptTemplate

# faiss——CPU 和 OpenMP 冲突问题
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

OLLAMA_URL = "http://localhost:11434"  # Ollama 默认地址
API_KEYS = {"OLLAMA_DEEPSEEK_API_KEY"}     # 自定义合法 API Key


# 定义示例数据
examples = [
  {"input":"happy","output":"sad"},
  {"input":"sad","output":"happy"},
  {"input":"tall","output":"short"},
  {"input":"short","output":"tall"},
  {"input":"angry","output":"excited"},
  {"input":"excited","output":"angry"},
  {"input":"开心","output":"悲伤"},
  {"input":"悲伤","output":"开心"},
]

# 创建提示词模板
example_prompt = PromptTemplate(
  input_variables=["input","output"],
  template="原词: {input} \n反义: {output}",
)

# 调用 Ollama API
embed = OllamaEmbeddings(
    model="deepseek-r1:1.5b",
    base_url=OLLAMA_URL,
)
# 创建一个示例选择器
example_selector = SemanticSimilarityExampleSelector.from_examples(
  # 传入示例
  examples=examples,
  # 相似性搜索
  embeddings=embed,
  # 使用的向量存储
  vectorstore_cls=Chroma,
  # 结果条数
  k=2,
)

mmr_prompt = FewShotPromptTemplate(
  example_selector=example_selector,
  example_prompt=example_prompt,
  prefix="给出每一个输入词的反义词: ",
  suffix="原词: {adjective} \n反义: ",
  input_variables=["adjective"]
)

# 当我们输入一个词时，我们会得到一个提示，提示我们输入这个词的反义词
prompt = mmr_prompt.format(adjective="担心")
print(prompt)

# # 执行链式调用
# from langchain_ollama import ChatOllama
# # 指定本地模型名称或路径
# ollama_llm = ChatOllama(
#     model="deepseek-r1:1.5b",  # 例如DeepSeek-R1 1.5B模型
#     base_url="http://localhost:11434"  # Ollama默认地址
# )
# response = ollama_llm.invoke(prompt)
# print(response)


给出每一个输入词的反义词: 

原词: 开心 
反义: 悲伤

原词: 悲伤 
反义: 开心

原词: 担心 
反义: 


## 总结
长度选择，根据约束输入长度来截取合适的示例
相似度示例选择器通过相似度选择来选择输入信息最接近的示例来用作提示词。